In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import cv2
from tqdm import tqdm

In [2]:
data_dir = os.path.join(os.getcwd(), 'data')

img_dir = os.path.join(data_dir, 'images_test') #hfusg images of atopic dermatitis
mask_dir = os.path.join(data_dir, 'masks_test') #masks of SLEB layer

IMG_WIDTH, IMG_HEIGHT = 256, 256

In [3]:
X = []

#looping through image files
for file in tqdm(os.listdir(img_dir)):
    
    fullpath = os.path.join(img_dir, file)
    img = cv2.imread(fullpath)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #resize images to the size of network input
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    X.append(img)
    
print(img.shape)

100%|█████████████████████████████████████████████████████████████████████████████| 3493/3493 [00:03<00:00, 961.69it/s]


(256, 256, 3)


In [4]:
y = []

#looping through mask files
for file in tqdm(os.listdir(mask_dir)):
    
    fullpath = os.path.join(mask_dir, file)
    #getting into .mat file from matlab, where masks are storaged
    mat_file =  sio.loadmat(fullpath)
    mat_e= mat_file['E']
    mat_e= mat_e['e']
    mat_arrs = mat_e[0].ravel()
    arrs = mat_arrs[0]
    arr = arrs[0,0]['azs']
    #resize images to the size of network input
    arr = cv2.resize(arr, (IMG_WIDTH, IMG_HEIGHT))
    y.append(arr)

print(arr.shape)

100%|█████████████████████████████████████████████████████████████████████████████| 3493/3493 [00:33<00:00, 105.15it/s]


(256, 256)


In [5]:
X, y = np.array(X), np.array(y)

print(X.shape)
print(y.shape)

(3493, 256, 256, 3)
(3493, 256, 256)


In [6]:
y = y.reshape((3493,256,256,1))

print(X.shape)
print(y.shape)

(3493, 256, 256, 3)
(3493, 256, 256, 1)


In [7]:
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.layers.core import Dropout, Lambda
from tensorflow.python.keras.layers.convolutional import Conv2D, Conv2DTranspose
from tensorflow.python.keras.layers.pooling import MaxPooling2D
from tensorflow.python.keras.layers.merge import concatenate
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.python.keras import backend as K
from tensorflow.keras import optimizers
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold


C:\Users\WD-X\Anaconda3\envs\tfgpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\WD-X\Anaconda3\envs\tfgpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\WD-X\Anaconda3\envs\tfgpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\WD-X\Anaconda3\envs\tfgpu\lib\site-packages\tensorflow\python\frame

In [8]:
smooth = 1.

#metric function Intersection over Union IoU (Jaccard Index)
def mean_iou(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou


In [9]:
inputs = Input((IMG_HEIGHT, IMG_WIDTH,  3))

c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (inputs)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.1) (c9)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

#output is a mask, size of 256x256
outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[mean_iou])
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 256, 256, 16) 0           conv2d[0][0]                     
________________________________________________________________________________

__________________________________________________________________________________________________
conv2d_transpose_3 (Conv2DTrans (None, 256, 256, 16) 2064        conv2d_15[0][0]                  
__________________________________________________________________________________________________
concatenate_3 (Concatenate)     (None, 256, 256, 32) 0           conv2d_transpose_3[0][0]         
                                                                 conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, 256, 256, 16) 4624        concatenate_3[0][0]              
__________________________________________________________________________________________________
dropout_8 (Dropout)             (None, 256, 256, 16) 0           conv2d_16[0][0]                  
__________________________________________________________________________________________________
conv2d_17 

In [11]:
#path for output network model
model_path = os.getcwd()+ '/unet.h5'

#checking learning progress
checkpoint = ModelCheckpoint(model_path,
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

#early stoppage of epoch
earlystop = EarlyStopping(monitor = "val_loss", #val_loss 
                          min_delta = 0, 
                          patience = 8,
                          mode="min",
                          verbose = 1,
                          restore_best_weights = True)

#reducing learning rate value 
reducelr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.2,
                              patience=3, 
                              min_lr=0.0001, 
                              verbose=1)


#cross-validation setup
train_X = X
train_y = y
n_folds = 4 #determining how many folds will be created
history = []

from keras.preprocessing.image import ImageDataGenerator
    
skf = KFold(n_splits = n_folds)
for train_index, test_index in skf.split(train_X):
    X_train, X_test = train_X[train_index], train_X[test_index]
    y_train, y_test = train_y[train_index], train_y[test_index]

#data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,
        featurewise_std_normalization=False,
        rotation_range=20,
        width_shift_range=0,
        height_shift_range=0,
        horizontal_flip=True)

    datagen.fit(X_train)

# fits the model on batches with real-time data augmentation:
    results= model.fit_generator(datagen.flow(X_train, y_train, batch_size=5),callbacks=[earlystop, checkpoint, reducelr],
                    steps_per_epoch=len(X_train), epochs=20,validation_data=(X_test, y_test))
    history.append(results.history)
    
#    results = model.fit(X_train, y_train,
#                    batch_size=5, epochs=100, 
#                    callbacks=[earlystop, checkpoint, reducelr],
#                    validation_data=(X_test, y_test))
#    history.append(results.history)

Epoch 1/20


 177/2619 [=>............................] - ETA: 5:13:13 - loss: 28.3379 - mean_iou: 0.18 - ETA: 2:43:10 - loss: 22.6338 - mean_iou: 0.14 - ETA: 1:53:14 - loss: 18.4275 - mean_iou: 0.12 - ETA: 1:28:15 - loss: 15.3133 - mean_iou: 0.11 - ETA: 1:13:18 - loss: 13.1366 - mean_iou: 0.10 - ETA: 1:03:17 - loss: 12.1084 - mean_iou: 0.09 - ETA: 56:10 - loss: 11.0266 - mean_iou: 0.0836 - ETA: 50:49 - loss: 10.2432 - mean_iou: 0.07 - ETA: 46:38 - loss: 9.5542 - mean_iou: 0.0716 - ETA: 43:17 - loss: 8.9586 - mean_iou: 0.069 - ETA: 40:34 - loss: 8.4135 - mean_iou: 0.070 - ETA: 38:17 - loss: 7.9211 - mean_iou: 0.071 - ETA: 36:23 - loss: 7.4858 - mean_iou: 0.072 - ETA: 34:43 - loss: 7.1040 - mean_iou: 0.073 - ETA: 33:17 - loss: 6.7680 - mean_iou: 0.075 - ETA: 32:02 - loss: 6.4572 - mean_iou: 0.075 - ETA: 30:55 - loss: 6.1694 - mean_iou: 0.075 - ETA: 29:56 - loss: 5.9287 - mean_iou: 0.077 - ETA: 29:03 - loss: 5.6939 - mean_iou: 0.079 - ETA: 28:15 - loss: 5.4884 - mean_iou: 0.080 - ETA: 27:32 - loss: 5

 355/2619 [===>..........................] - ETA: 14:05 - loss: 1.0814 - mean_iou: 0.090 - ETA: 14:04 - loss: 1.0788 - mean_iou: 0.091 - ETA: 14:03 - loss: 1.0747 - mean_iou: 0.090 - ETA: 14:02 - loss: 1.0708 - mean_iou: 0.090 - ETA: 14:01 - loss: 1.0675 - mean_iou: 0.090 - ETA: 14:01 - loss: 1.0635 - mean_iou: 0.090 - ETA: 14:00 - loss: 1.0596 - mean_iou: 0.090 - ETA: 13:59 - loss: 1.0564 - mean_iou: 0.090 - ETA: 13:58 - loss: 1.0525 - mean_iou: 0.090 - ETA: 13:58 - loss: 1.0489 - mean_iou: 0.090 - ETA: 13:57 - loss: 1.0462 - mean_iou: 0.090 - ETA: 13:56 - loss: 1.0436 - mean_iou: 0.091 - ETA: 13:56 - loss: 1.0407 - mean_iou: 0.091 - ETA: 13:55 - loss: 1.0371 - mean_iou: 0.091 - ETA: 13:54 - loss: 1.0343 - mean_iou: 0.091 - ETA: 13:53 - loss: 1.0321 - mean_iou: 0.091 - ETA: 13:53 - loss: 1.0296 - mean_iou: 0.091 - ETA: 13:52 - loss: 1.0279 - mean_iou: 0.092 - ETA: 13:51 - loss: 1.0253 - mean_iou: 0.092 - ETA: 13:51 - loss: 1.0228 - mean_iou: 0.092 - ETA: 13:50 - loss: 1.0197 - mean_io

 533/2619 [=====>........................] - ETA: 12:56 - loss: 0.7603 - mean_iou: 0.093 - ETA: 12:55 - loss: 0.7594 - mean_iou: 0.093 - ETA: 12:55 - loss: 0.7585 - mean_iou: 0.093 - ETA: 12:54 - loss: 0.7573 - mean_iou: 0.093 - ETA: 12:54 - loss: 0.7566 - mean_iou: 0.093 - ETA: 12:54 - loss: 0.7555 - mean_iou: 0.093 - ETA: 12:53 - loss: 0.7544 - mean_iou: 0.093 - ETA: 12:53 - loss: 0.7536 - mean_iou: 0.093 - ETA: 12:52 - loss: 0.7529 - mean_iou: 0.093 - ETA: 12:52 - loss: 0.7519 - mean_iou: 0.094 - ETA: 12:52 - loss: 0.7510 - mean_iou: 0.094 - ETA: 12:51 - loss: 0.7510 - mean_iou: 0.094 - ETA: 12:51 - loss: 0.7500 - mean_iou: 0.094 - ETA: 12:50 - loss: 0.7490 - mean_iou: 0.094 - ETA: 12:50 - loss: 0.7482 - mean_iou: 0.094 - ETA: 12:50 - loss: 0.7474 - mean_iou: 0.094 - ETA: 12:49 - loss: 0.7464 - mean_iou: 0.093 - ETA: 12:49 - loss: 0.7459 - mean_iou: 0.094 - ETA: 12:48 - loss: 0.7451 - mean_iou: 0.094 - ETA: 12:48 - loss: 0.7444 - mean_iou: 0.094 - ETA: 12:48 - loss: 0.7434 - mean_io

 711/2619 [=======>......................] - ETA: 11:49 - loss: 0.6434 - mean_iou: 0.103 - ETA: 11:49 - loss: 0.6428 - mean_iou: 0.103 - ETA: 11:48 - loss: 0.6422 - mean_iou: 0.103 - ETA: 11:48 - loss: 0.6418 - mean_iou: 0.103 - ETA: 11:48 - loss: 0.6415 - mean_iou: 0.103 - ETA: 11:47 - loss: 0.6415 - mean_iou: 0.103 - ETA: 11:47 - loss: 0.6409 - mean_iou: 0.103 - ETA: 11:47 - loss: 0.6404 - mean_iou: 0.103 - ETA: 11:46 - loss: 0.6398 - mean_iou: 0.103 - ETA: 11:46 - loss: 0.6394 - mean_iou: 0.104 - ETA: 11:46 - loss: 0.6391 - mean_iou: 0.104 - ETA: 11:45 - loss: 0.6385 - mean_iou: 0.104 - ETA: 11:45 - loss: 0.6380 - mean_iou: 0.104 - ETA: 11:44 - loss: 0.6376 - mean_iou: 0.104 - ETA: 11:44 - loss: 0.6374 - mean_iou: 0.104 - ETA: 11:44 - loss: 0.6370 - mean_iou: 0.104 - ETA: 11:43 - loss: 0.6366 - mean_iou: 0.104 - ETA: 11:43 - loss: 0.6362 - mean_iou: 0.104 - ETA: 11:43 - loss: 0.6356 - mean_iou: 0.104 - ETA: 11:42 - loss: 0.6353 - mean_iou: 0.104 - ETA: 11:42 - loss: 0.6353 - mean_io

...

2009/2619 [======================>.......] - ETA: 4:24 - loss: 0.1740 - mean_iou: 0.40 - ETA: 4:24 - loss: 0.1740 - mean_iou: 0.40 - ETA: 4:23 - loss: 0.1740 - mean_iou: 0.40 - ETA: 4:23 - loss: 0.1740 - mean_iou: 0.40 - ETA: 4:23 - loss: 0.1740 - mean_iou: 0.40 - ETA: 4:22 - loss: 0.1740 - mean_iou: 0.40 - ETA: 4:22 - loss: 0.1741 - mean_iou: 0.40 - ETA: 4:22 - loss: 0.1741 - mean_iou: 0.40 - ETA: 4:21 - loss: 0.1740 - mean_iou: 0.40 - ETA: 4:21 - loss: 0.1741 - mean_iou: 0.40 - ETA: 4:21 - loss: 0.1741 - mean_iou: 0.40 - ETA: 4:20 - loss: 0.1741 - mean_iou: 0.40 - ETA: 4:20 - loss: 0.1740 - mean_iou: 0.40 - ETA: 4:20 - loss: 0.1740 - mean_iou: 0.40 - ETA: 4:19 - loss: 0.1741 - mean_iou: 0.40 - ETA: 4:19 - loss: 0.1741 - mean_iou: 0.40 - ETA: 4:19 - loss: 0.1741 - mean_iou: 0.40 - ETA: 4:18 - loss: 0.1741 - mean_iou: 0.40 - ETA: 4:18 - loss: 0.1741 - mean_iou: 0.40 - ETA: 4:18 - loss: 0.1741 - mean_iou: 0.40 - ETA: 4:17 - loss: 0.1741 - mean_iou: 0.40 - ETA: 4:17 - loss: 0.1740 - mean

2195/2619 [========================>.....] - ETA: 3:23 - loss: 0.1737 - mean_iou: 0.40 - ETA: 3:22 - loss: 0.1737 - mean_iou: 0.40 - ETA: 3:22 - loss: 0.1737 - mean_iou: 0.40 - ETA: 3:22 - loss: 0.1737 - mean_iou: 0.40 - ETA: 3:21 - loss: 0.1737 - mean_iou: 0.40 - ETA: 3:21 - loss: 0.1737 - mean_iou: 0.40 - ETA: 3:21 - loss: 0.1737 - mean_iou: 0.40 - ETA: 3:20 - loss: 0.1737 - mean_iou: 0.40 - ETA: 3:20 - loss: 0.1737 - mean_iou: 0.40 - ETA: 3:20 - loss: 0.1737 - mean_iou: 0.40 - ETA: 3:19 - loss: 0.1737 - mean_iou: 0.40 - ETA: 3:19 - loss: 0.1737 - mean_iou: 0.40 - ETA: 3:19 - loss: 0.1737 - mean_iou: 0.40 - ETA: 3:18 - loss: 0.1737 - mean_iou: 0.40 - ETA: 3:18 - loss: 0.1737 - mean_iou: 0.40 - ETA: 3:18 - loss: 0.1736 - mean_iou: 0.40 - ETA: 3:17 - loss: 0.1736 - mean_iou: 0.40 - ETA: 3:17 - loss: 0.1736 - mean_iou: 0.40 - ETA: 3:17 - loss: 0.1736 - mean_iou: 0.40 - ETA: 3:16 - loss: 0.1736 - mean_iou: 0.40 - ETA: 3:16 - loss: 0.1736 - mean_iou: 0.40 - ETA: 3:16 - loss: 0.1736 - mean

2381/2619 [==========================>...] - ETA: 2:21 - loss: 0.1732 - mean_iou: 0.40 - ETA: 2:20 - loss: 0.1732 - mean_iou: 0.40 - ETA: 2:20 - loss: 0.1732 - mean_iou: 0.40 - ETA: 2:20 - loss: 0.1732 - mean_iou: 0.40 - ETA: 2:19 - loss: 0.1732 - mean_iou: 0.40 - ETA: 2:19 - loss: 0.1732 - mean_iou: 0.40 - ETA: 2:19 - loss: 0.1732 - mean_iou: 0.40 - ETA: 2:18 - loss: 0.1732 - mean_iou: 0.40 - ETA: 2:18 - loss: 0.1732 - mean_iou: 0.40 - ETA: 2:18 - loss: 0.1732 - mean_iou: 0.40 - ETA: 2:17 - loss: 0.1732 - mean_iou: 0.40 - ETA: 2:17 - loss: 0.1732 - mean_iou: 0.40 - ETA: 2:17 - loss: 0.1732 - mean_iou: 0.40 - ETA: 2:16 - loss: 0.1732 - mean_iou: 0.40 - ETA: 2:16 - loss: 0.1732 - mean_iou: 0.40 - ETA: 2:16 - loss: 0.1732 - mean_iou: 0.40 - ETA: 2:15 - loss: 0.1731 - mean_iou: 0.40 - ETA: 2:15 - loss: 0.1731 - mean_iou: 0.40 - ETA: 2:15 - loss: 0.1731 - mean_iou: 0.40 - ETA: 2:14 - loss: 0.1731 - mean_iou: 0.40 - ETA: 2:14 - loss: 0.1731 - mean_iou: 0.40 - ETA: 2:14 - loss: 0.1731 - mean

2463/2619 [===========================>..] - ETA: 1:19 - loss: 0.1728 - mean_iou: 0.40 - ETA: 1:18 - loss: 0.1727 - mean_iou: 0.40 - ETA: 1:18 - loss: 0.1727 - mean_iou: 0.40 - ETA: 1:18 - loss: 0.1727 - mean_iou: 0.40 - ETA: 1:17 - loss: 0.1727 - mean_iou: 0.40 - ETA: 1:17 - loss: 0.1727 - mean_iou: 0.40 - ETA: 1:17 - loss: 0.1728 - mean_iou: 0.40 - ETA: 1:16 - loss: 0.1728 - mean_iou: 0.40 - ETA: 1:16 - loss: 0.1728 - mean_iou: 0.40 - ETA: 1:16 - loss: 0.1727 - mean_iou: 0.40 - ETA: 1:15 - loss: 0.1727 - mean_iou: 0.40 - ETA: 1:15 - loss: 0.1727 - mean_iou: 0.40 - ETA: 1:15 - loss: 0.1727 - mean_iou: 0.40 - ETA: 1:14 - loss: 0.1727 - mean_iou: 0.40 - ETA: 1:14 - loss: 0.1727 - mean_iou: 0.40 - ETA: 1:14 - loss: 0.1727 - mean_iou: 0.40 - ETA: 1:13 - loss: 0.1727 - mean_iou: 0.40 - ETA: 1:13 - loss: 0.1727 - mean_iou: 0.40 - ETA: 1:13 - loss: 0.1726 - mean_iou: 0.40 - ETA: 1:12 - loss: 0.1727 - mean_iou: 0.40 - ETA: 1:12 - loss: 0.1727 - mean_iou: 0.40 - ETA: 1:12 - loss: 0.1727 - mean

KeyboardInterrupt: 

In [ ]:
from keras.utils.vis_utils import plot_model


model_dict = history[0]
model_dict1 = history[1]
model_dict2 = history[2]
model_dict3 = history[3]

loss_values = model_dict['loss']
loss_values.extend(model_dict1['loss'])
loss_values.extend(model_dict2['loss'])
loss_values.extend(model_dict3['loss'])


mean_iou_values = model_dict['mean_iou']
mean_iou_values.extend(model_dict1['mean_iou'])
mean_iou_values.extend(model_dict2['mean_iou'])
mean_iou_values.extend(model_dict3['mean_iou'])
epochs_as_list = range(1, len(loss_values) +1)

print(type(epochs_as_list))

#plotting learning history
plt.figure(figsize=(20,10))
plt.style.use('bmh')

train_loss_line = plt.plot(epochs_as_list, loss_values, label = 'Train Loss' )
test_loss_line = plt.plot(epochs_as_list, mean_iou_values, label = 'Mean IoU' )

plt.setp(train_loss_line, linewidth = 2.0, marker ='o', markersize = 5.0)
plt.setp(test_loss_line, linewidth = 2.0, marker ='o', markersize = 5.0)

plt.xlabel('Epochs')
plt.ylabel('Loss/IoU')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
x = X[100]
x.shape

In [ ]:
#image check
plt.figure(figsize=(10,10))
plt.imshow(x, cmap='gray')
plt.axis('off')


In [ ]:
#network prediction of SLEB layer
x = x.reshape(1,256,256,3)
pred = model.predict(x)
pred = pred.reshape(256,256)
print(np.unique(pred))

for w in range(0,256):
    for h  in range(0,256):
        
        if pred[w,h] > 0.9: #threshold of propability matrix, range [0,1].
                            #1 is 100% sureness of network pixel belongs to SLEB layer
            pred[w,h] = 1
        else:
            pred[w,h] = 0
plt.figure(figsize=(10,10))
plt.imshow(pred, cmap='gray')
plt.axis('off')
